In [3]:
#pip install gensim
#!pip install tokenizers
from pathlib import Path
save_path = 'tokenized_data'

In [4]:
import tensorflow as tf
import transformers
from transformers import GPT2Config
from transformers import TFGPT2LMHeadModel
from transformers import GPT2Tokenizer

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained(save_path)
tokenizer.add_special_tokens({
  "eos_token": "</s>",
  "bos_token": "<s>",
  "unk_token": "<unk>",
  "pad_token": "<pad>",
  "mask_token": "<mask>"
})# creating the configurations from which the model can be made
config = GPT2Config(
  vocab_size=tokenizer.vocab_size,
  bos_token_id=tokenizer.bos_token_id,
  eos_token_id=tokenizer.eos_token_id
)# creating the model
model = TFGPT2LMHeadModel(config)

In [6]:
with open('single_string.txt', "r", encoding='utf-8') as f:
    single_string = f.read()

In [14]:
chunks = [single_string[i:i + 10000000] for i in range(0, len(single_string), 10000000)]

In [15]:
len(chunks), len(chunks[0])

(497, 10000000)

In [ ]:
token_list = []
for i,chunk in enumerate(chunks):
    print("chunk:",i)
    token_list += tokenizer.encode(chunk)

chunk: 0
chunk: 1
chunk: 2
chunk: 3
chunk: 4
chunk: 5
chunk: 6
chunk: 7
chunk: 8
chunk: 9
chunk: 10
chunk: 11
chunk: 12
chunk: 13
chunk: 14
chunk: 15
chunk: 16
chunk: 17
chunk: 18
chunk: 19
chunk: 20
chunk: 21
chunk: 22
chunk: 23
chunk: 24
chunk: 25
chunk: 26
chunk: 27
chunk: 28
chunk: 29
chunk: 30
chunk: 31
chunk: 32
chunk: 33
chunk: 34
chunk: 35
chunk: 36
chunk: 37
chunk: 38
chunk: 39
chunk: 40
chunk: 41
chunk: 42
chunk: 43
chunk: 44
chunk: 45
chunk: 46
chunk: 47
chunk: 48
chunk: 49
chunk: 50
chunk: 51
chunk: 52
chunk: 53
chunk: 54
chunk: 55
chunk: 56
chunk: 57
chunk: 58
chunk: 59
chunk: 60
chunk: 61
chunk: 62
chunk: 63
chunk: 64
chunk: 65
chunk: 66
chunk: 67
chunk: 68
chunk: 69
chunk: 70
chunk: 71
chunk: 72
chunk: 73
chunk: 74
chunk: 75
chunk: 76
chunk: 77
chunk: 78
chunk: 79
chunk: 80
chunk: 81
chunk: 82
chunk: 83
chunk: 84
chunk: 85
chunk: 86
chunk: 87
chunk: 88
chunk: 89
chunk: 90
chunk: 91
chunk: 92
chunk: 93
chunk: 94
chunk: 95
chunk: 96
chunk: 97
chunk: 98
chunk: 99
chunk: 100

In [25]:
len(token_list)

1087269997

In [31]:
tokenizer.decode(token_list[:100])

"Corteza de ''E. fibrosa''. '''Corteza de hierro''' (Ironbark) es el nombre que se le da varias especies en tres grupos taxonómicos dentro del género de ''Eucalyptus'' que tienen la corteza oscura profundamente surcada. En vez de mudarla anualmente como en muchas otras especies de ''Eucalyptus'', la corteza muerta se acumula en los árboles, formando las fisuras. Se hace rugosa después de resecarse impregnarse con k"

In [32]:
import pickle
with open('token_list.data', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(token_list, filehandle)

In [33]:
#with open('token_list.data', 'rb') as filehandle:
#    token_list = pickle.load(filehandle)

In [35]:
%%time
examples = []
block_size = 1000
BATCH_SIZE = 16
BUFFER_SIZE = 1000

for i in range(0, len(token_list) - block_size + 1, block_size):
    examples.append(token_list[i:i + block_size])

CPU times: user 29.3 s, sys: 100 ms, total: 29.4 s
Wall time: 29.4 s


In [36]:
#examples = [token_list[i:i + block_size] for i in range(0, len(token_list), block_size)]

In [37]:
len(examples), len(examples[0])

(1087269, 1000)

In [38]:
inputs, labels = [], []

for ex in examples:
    inputs.append(ex[:-1])
    labels.append(ex[1:])

In [41]:
GPUs =tf.config.experimental.list_physical_devices('GPU')

In [42]:
GPUs[0]

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')

In [ ]:
tf.debugging.set_log_device_placement(True)

strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
    dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


In [ ]:
#dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
#dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
tf.data.experimental.save(dataset, './tensorflow_dataset', compression=None, shard_func=None)

In [ ]:
while 1:
    time.sleep(1)
    print('hola')

# Model Training

In [ ]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)

# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])